In [1]:
import pandas as pd

In [2]:
import sys;
import select;
import socket;
import time;
import os;
import numpy as np
import random;
import ctypes;
from time import sleep;
import time;


In [3]:
import ftd3xx
import sys
import ftd3xx._ftd3xx_linux as _ft

In [44]:
print(sys.platform)
def ReadPipe(D3XX, pipe, size):
    print("Read:",pipe-0x82)
    return D3XX.readPipeEx(pipe-0x82, size)

def CancelPipe(D3XX, pipe):

    if sys.platform == 'linux':
        return D3XX.flushPipe(pipe)

    return D3XX.abortPipe(pipe)

linux


In [5]:
numDevices = ftd3xx.createDeviceInfoList()
print(numDevices)
if (numDevices == 0):
    print("ERROR: No FTD3XX device is detected.");
else:
    print(numDevices," Device")
devList = ftd3xx.getDeviceInfoList()
print(devList)
# Just open the first device (index 0)
devIndex = 0;
D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);  

1
1  Device


In [6]:
cfg = D3XX.getChipConfiguration();
print("CLK: ",cfg.FIFOClock)
print("Mode: ",cfg.FIFOMode)
print("FlashEEPROMDetection: ",cfg.FlashEEPROMDetection)
VP = D3XX.getVIDPID()
print("VID/PID:",hex(VP[0]), hex(VP[1]))

CLK:  0
Mode:  0
FlashEEPROMDetection:  16
VID/PID: 0x60 0x1e


In [7]:
# Reset the FT600 to make sure starting fresh with nothing in FIFOs
D3XX.resetDevicePort(); # Flush
D3XX.close();

In [8]:
D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);

# check if USB3 or USB2
devDesc = D3XX.getDeviceDescriptor();
bUSB3 = devDesc.bcdUSB >= 0x300;

# validate chip configuration
cfg = D3XX.getChipConfiguration();
cfg.FIFOClock

0

In [9]:
D3XX.setChipConfiguration(cfg);

In [10]:
desired_freq = 1;# 1=66 MHz, 0=100 MHz
if ( cfg.FIFOClock != desired_freq ):
    print("Changing FT600 Frequency");
    cfg.FIFOClock = desired_freq;
    D3XX.setChipConfiguration(cfg);
D3XX.resetDevicePort(); # Flush
D3XX.close();

Changing FT600 Frequency


In [11]:
numDevices = ftd3xx.createDeviceInfoList()
if (numDevices == 0):
    print("ERROR: No FTD3XX device is detected.");
else:
    print(numDevices," Device")
devList = ftd3xx.getDeviceInfoList()
print(devList)
devIndex = 0;
D3XX = ftd3xx.create(devIndex, _ft.FT_OPEN_BY_INDEX);

1  Device


In [12]:
cfg = D3XX.getChipConfiguration();
cfg.FIFOClock

cfg = D3XX.getChipConfiguration();
print("\tVendorID = %#06x" % cfg.VendorID)
print("\tProductID = %#06x" % cfg.ProductID)
cfg.FIFOMode,D3XX.getFirmwareVersion()

	VendorID = 0x0403
	ProductID = 0x601e


(0, 265)

In [13]:
import ctypes
conf = _ft.FT_TRANSFER_CONF();
conf.wStructSize = ctypes.sizeof(_ft.FT_TRANSFER_CONF);
conf.pipe[_ft.FT_PIPE_DIR_IN].fPipeNotUsed = True;
conf.pipe[_ft.FT_PIPE_DIR_OUT].fPipeNotUsed = True;
conf.pipe.fReserved = False;
conf.pipe.fKeepDeviceSideBufferAfterReopen = False;
for i in range(4):
    ftd3xx.setTransferParams(conf, i);

In [14]:
print(sys.platform)
def ReadPipe(D3XX, pipe, size):

    if sys.platform == 'linux':
        pipe -= 0x82
    print("Pipe, ",pipe)
    return D3XX.readPipeEx(pipe, size)

def CancelPipe(D3XX, pipe):

    if sys.platform == 'linux':
        return D3XX.flushPipe(pipe)

    return D3XX.abortPipe(pipe)

linux


In [15]:
# Call before FT_Create when non-transfer functions will be called
# Only needed for RevA chip (Firmware 1.0.2)
# Not necessary starting RevB chip (Firmware 1.0.9)
conf = _ft.FT_TRANSFER_CONF();
conf.wStructSize = ctypes.sizeof(_ft.FT_TRANSFER_CONF);
conf.pipe[_ft.FT_PIPE_DIR_IN].fPipeNotUsed = True;
conf.pipe[_ft.FT_PIPE_DIR_OUT].fPipeNotUsed = True;
conf.pipe.fReserved = False;
conf.pipe.fKeepDeviceSideBufferAfterReopen = False;
for i in range(4):
    ftd3xx.setTransferParams(conf, i);

In [16]:
D3XX.getLastError()

0

In [56]:
numChannels = [4, 2, 1, 0, 0]
numChannels = numChannels[cfg.ChannelConfig]
numChannels

1

In [18]:
A = []
N = 4
start_time = time.time()
for i in range(N):
    A.append(ReadPipe(D3XX,  0x82, 4096*8*8))
stop = time.time()
print(stop-start_time)

Pipe,  0
Pipe,  0
Pipe,  0
Pipe,  0
4.005713939666748


In [19]:
A[-1]['bytesTransferred']*N/(stop-start_time)

0.0

In [45]:
start_time = time.time()
A.append(ReadPipe(D3XX,  0x82, 500000000))
stop = time.time()
print(stop-start_time)

Read: 0
1.302826166152954


In [21]:
500000000 / (stop-start_time)

381164253.3466231

```
T:  Bus=02 Lev=01 Prnt=01 Port=03 Cnt=01 Dev#=  3 Spd=480 MxCh= 0
D:  Ver= 2.10 Cls=ef(misc ) Sub=02 Prot=01 MxPS=64 #Cfgs=  1
P:  Vendor=0403 ProdID=601e Rev=00.00
S:  Manufacturer=FTDI
S:  Product=FTDI SuperSpeed-FIFO Bridge
S:  SerialNumber=000000000001
C:  #Ifs= 2 Cfg#= 1 Atr=e0 MxPwr=96mA
I:  If#= 0 Alt= 0 #EPs= 2 Cls=ff(vend.) Sub=ff Prot=ff Driver=usbfs
E:  Ad=01(O) Atr=02(Bulk) MxPS= 512 Ivl=0ms
E:  Ad=81(I) Atr=03(Int.) MxPS=  64 Ivl=32ms
I:  If#= 1 Alt= 0 #EPs= 2 Cls=ff(vend.) Sub=ff Prot=ff Driver=usbfs
E:  Ad=02(O) Atr=02(Bulk) MxPS= 512 Ivl=0ms
E:  Ad=82(I) Atr=02(Bulk) MxPS= 512 Ivl=0ms
```



```
Bus 002 Device 003: ID 0403:601e Future Technology Devices International, Ltd FTDI SuperSpeed-FIFO Bridge
Device Descriptor:
  bLength                18
  bDescriptorType         1
  bcdUSB               2.10
  bDeviceClass          239 Miscellaneous Device
  bDeviceSubClass         2 
  bDeviceProtocol         1 Interface Association
  bMaxPacketSize0        64
  idVendor           0x0403 Future Technology Devices International, Ltd
  idProduct          0x601e 
  bcdDevice            0.00
  iManufacturer           1 FTDI
  iProduct                2 FTDI SuperSpeed-FIFO Bridge
  iSerial                 3 000000000001
  bNumConfigurations      1
  Configuration Descriptor:
    bLength                 9
    bDescriptorType         2
    wTotalLength       0x003f
    bNumInterfaces          2
    bConfigurationValue     1
    iConfiguration          0 
    bmAttributes         0xe0
      Self Powered
      Remote Wakeup
    MaxPower               96mA
    Interface Association:
      bLength                 8
      bDescriptorType        11
      bFirstInterface         0
      bInterfaceCount         2
      bFunctionClass        255 Vendor Specific Class
      bFunctionSubClass     255 Vendor Specific Subclass
      bFunctionProtocol     255 Vendor Specific Protocol
      iFunction               0 
    Interface Descriptor:
      bLength                 9
      bDescriptorType         4
      bInterfaceNumber        0
      bAlternateSetting       0
      bNumEndpoints           2
      bInterfaceClass       255 Vendor Specific Class
      bInterfaceSubClass    255 Vendor Specific Subclass
      bInterfaceProtocol    255 Vendor Specific Protocol
      iInterface              0 
      Endpoint Descriptor:
        bLength                 7
        bDescriptorType         5
        bEndpointAddress     0x01  EP 1 OUT
        bmAttributes            2
          Transfer Type            Bulk
          Synch Type               None
          Usage Type               Data
        wMaxPacketSize     0x0200  1x 512 bytes
        bInterval               0
      Endpoint Descriptor:
        bLength                 7
        bDescriptorType         5
        bEndpointAddress     0x81  EP 1 IN
        bmAttributes            3
          Transfer Type            Interrupt
          Synch Type               None
          Usage Type               Data
        wMaxPacketSize     0x0040  1x 64 bytes
        bInterval               9
    Interface Descriptor:
      bLength                 9
      bDescriptorType         4
      bInterfaceNumber        1
      bAlternateSetting       0
      bNumEndpoints           2
      bInterfaceClass       255 Vendor Specific Class
      bInterfaceSubClass    255 Vendor Specific Subclass
      bInterfaceProtocol    255 Vendor Specific Protocol
      iInterface              0 
      Endpoint Descriptor:
        bLength                 7
        bDescriptorType         5
        bEndpointAddress     0x02  EP 2 OUT
        bmAttributes            2
          Transfer Type            Bulk
          Synch Type               None
          Usage Type               Data
        wMaxPacketSize     0x0200  1x 512 bytes
        bInterval               0
      Endpoint Descriptor:
        bLength                 7
        bDescriptorType         5
        bEndpointAddress     0x82  EP 2 IN
        bmAttributes            2
          Transfer Type            Bulk
          Synch Type               None
          Usage Type               Data
        wMaxPacketSize     0x0200  1x 512 bytes
        bInterval               0
Binary Object Store Descriptor:
  bLength                 5
  bDescriptorType        15
  wTotalLength       0x0016
  bNumDeviceCaps          2
  USB 2.0 Extension Device Capability:
    bLength                 7
    bDescriptorType        16
    bDevCapabilityType      2
    bmAttributes   0x00000006
      BESL Link Power Management (LPM) Supported
  SuperSpeed USB Device Capability:
    bLength                10
    bDescriptorType        16
    bDevCapabilityType      3
    bmAttributes         0x00
    wSpeedsSupported   0x000e
      Device can operate at Full Speed (12Mbps)
      Device can operate at High Speed (480Mbps)
      Device can operate at SuperSpeed (5Gbps)
    bFunctionalitySupport   2
      Lowest fully-functional device speed is High Speed (480Mbps)
    bU1DevExitLat          10 micro seconds
    bU2DevExitLat        2047 micro seconds
Device Status:     0x0001
  Self Powered

```

In [49]:
def write(D3XX, pipe, bytes, timeout=1000):

    print(len(bytes))
    return D3XX.writePipe(pipe, bytes, len(bytes), timeout)


In [79]:
channel = 0

PIPE = 0x02 + channel
if sys.platform == 'linux':
    PIPE -= 0x02
    
B = bytes.fromhex("8100")
print(B,D3XX.writePipe(PIPE, B, len(B)))

b'\x81\x00' 2


In [84]:
D3XX.writePipe(PIPE, B, len(B))
messages = [0x1000,0x2080,0x3000]

for opcode in messages:
    B = opcode.to_bytes(2, 'little')
    print(D3XX.writePipe(PIPE, B, len(B)))

2
2
2


In [83]:
ReadPipe(D3XX,  0x82, 1000000)

Read: 0


{'bytesTransferred': 0, 'bytes': b''}